In [1]:
from os import listdir
from keras.models import model_from_json
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import sentence_bleu,corpus_bleu
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tqdm import tqdm
import numpy as np
import h5py as h5py
from compiler.classes.Compiler import *
from compiler.classes.Utils import *
from compiler.classes.Node import *

Using TensorFlow backend.


In [2]:
# Read a file and return a string
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def load_data(data_dir):
    text = []
    images = []
    # Load all the files and order them
    all_filenames = listdir(data_dir)
    all_filenames.sort()
    #print("i'here")
    for filename in (all_filenames)[-2:]:
        #print("i'm inside")
        if filename[-3:] == "npz":
            # Load the images already prepared in arrays
            image = np.load(data_dir+filename)
            images.append(image['features'])
        else:
            # Load the boostrap tokens and rap them in a start and end tag
            syntax = '<START> ' + load_doc(data_dir+filename) + ' <END>'
            # Seperate all the words with a single space
            syntax = ' '.join(syntax.split())
            # Add a space after each comma
            syntax = syntax.replace(',', ' ,')
            text.append(syntax)
    images = np.array(images, dtype=float)
    return images, text

In [9]:
# Initialize the function to create the vocabulary 
tokenizer = Tokenizer(filters='', split=" ", lower=False)
# Create the vocabulary 
#tokenizer.fit_on_texts([load_doc('Vocabulary/carbon.vocab')])
#tokenizer.fit_on_texts([load_doc('Vocabulary/carbon_A+B+D.vocab')])
tokenizer.fit_on_texts([load_doc('Vocabulary/carbon_A_Dataset.vocab')])


In [17]:
dir_name = './test_set_new/'
train_features, texts = load_data(dir_name)


In [5]:
#load model and weights 
json_file = open('A+B+D-june18-100.json', 'r')
#json_file = open('A-may15-100.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("A-june18-org-weights-epoch-0100--val_loss-3.5928--loss-0.4477.hdf5")
#loaded_model.load_weights("A-may15-org-weights-epoch-0050--val_loss-2.2828--loss-0.1636.hdf5")

print("Loaded model from disk")

Loaded model from disk


In [6]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
print(word_for_id(15, tokenizer))

context-switcher


In [7]:
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    photo = np.array([photo])
    # seed the generation process
    in_text = '<START> '
    # iterate over the whole length of the sequence
    print('\nPrediction---->\n\n<START> ', end='')
    for i in range(150):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = loaded_model.predict([photo, sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += word + ' '
        # stop if we predict the end of the sequence
        print(word + ' ', end='')
        if word == '<END>':
            break
    return in_text

In [18]:
max_length = 17

def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()

    for i in range(len(texts)):
        yhat = generate_desc(model, tokenizer, photos[i], max_length)
     
        actual.append([texts[i].split()])
        predicted.append(yhat.split())

    bleu = corpus_bleu(actual, predicted)
    return bleu, actual, predicted

bleu, actual, predicted = evaluate_model(loaded_model, texts, train_features, tokenizer, max_length)


Prediction---->

<START> search , btn-secondary , text-input-basic , Checkbox } <END> 

In [ ]:
from __future__ import print_function

import sys
from os.path import basename
from classes.Utils import *
from classes.Compiler import *

if (__name__ == "__main__"):
    argv = sys.argv[1:]
    length = len(argv)
    if length != 0:
        input_file = argv[0]
    else:
        print("Error: not enough argument supplied:")
        print("carbon-compiler.py <i")
        exit(0)


def render_content_with_text(key, value):
    if (FILL_WITH_RANDOM_TEXT):
        if (key.find("button") != -1):
            value = value.replace(TEXT_PLACE_HOLDER, Utils.get_random_text())
        
        elif (key.find("CodeSnippet") != -1):
            value = value.replace(TEXT_PLACE_HOLDER,
                                  Utils.get_random_text(length_text=30, space_number=7, with_upper_case=False))
            
        elif (key.find("Search") != -1):
            value = value.replace(TEXT_PLACE_HOLDER,
                                  Utils.get_random_text(length_text=35, space_number=7, with_upper_case=False))
    return value
    

#dsl_path = "assets/carbon-dsl-mapping.json"
dsl_path="dsl_mapper/dsl_mapper_updated.json"
compiler = Compiler(dsl_path)
FILL_WITH_RANDOM_TEXT = True
TEXT_PLACE_HOLDER = "[]"
file_uid = basename(input_file)[:basename(input_file).find(".")]
path = input_file[:input_file.find(file_uid)]
input_file_path = "{}{}.gui".format(path, file_uid)
output_file_path = "{}{}.xml".format(path, file_uid)
#compiler.compile("[ 'text-input-basic', ',', ',', 'text-input-basic', '}', 'text-input-basic', 'text-input-basic', 'text-input-basic', 'text-input-basic', ", output_file_path)
#compiler.compile( " text-input-basic , text-input-basic , text-input-basic  , accordionItem , btn-secondary , btn-primary " , output_file_path)
compiler.compile(predicted[0] , output_file_path)

In [25]:
print(bleu)

0.8239677834947428


In [22]:
print(predicted[0])

['<START>', 'form', '{', 'text-input-basic', ',', 'text-input-basic', ',', 'text-input-basic', ',', 'accordionItem', ',', 'accordionItem', ',', 'btn-primary', '}', '<END>']
